In [1]:
import pandas as pd
import numpy as np
import matplotlib
import torch
from ecog_speech import datasets, feature_processing, experiments
from ecog_speech.models import chang2020 as chm
from tqdm.auto import tqdm

%matplotlib inline

/home/morgan/.conda/envs/venv/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [3]:
example_data_map = datasets.NorthwesternWords.load_data(location='Mayo Clinic', patient=19, session=1, trial=1, 
                                                               # Will use package 'data/' directory
                                                               #base_path=/path/to/data/3-Vetteed Datasets/Single Word/
                                                              )

---19-1-1-Mayo Clinic---
Duplicate words (n=2): pig, cup
ECOG_signal shape: (422400, 96) [(Timedelta('0 days 00:00:00'), Timedelta('0 days 00:05:51.999166667'))]
Selected sensors (n=64): 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63
Audio shape: (16887643,) [(Timedelta('0 days 00:00:00'), Timedelta('0 days 00:05:51.825875'))]


**Use shift operator to easily compose a pipeline**

In [4]:
preproc_pipeline = feature_processing.WordStopStartTimeMap() >> feature_processing.PowerThreshold(threshold=0.0075)

In [5]:
preproc_pipeline

ProcessPipeline(name='ProcessPipeline', steps=[(WordStopStartTimeMap(name='WordStopStartTimeMap', input_override=None, input_remap=None, output_remap=None), {}), (PowerThreshold(name='PowerThreshold', input_override=None, input_remap=None, threshold=0.0075, output_remap={'stim': 'stim_pwrt', 'stim_diff': 'stim_pwrt_diff'}), {})])

In [7]:
preprocessed_data_map = preproc_pipeline(example_data_map)

Sample Words: 0-much,1-fish,2-ride,3-bit,4-bit
